flow of code 
    1-convert json into data structure
    2-helper function to closure of node
    3-transantions table
todo list
    1-coverting json => Done
    2-Get Closure    => Done
    3-Define DFA     => Done
    4-algorithm   


Check List in json    

In [2]:
from tokenize import String

In [3]:
import json

# Read the JSON file
with open('B:\\Spring2024\\Compilers\\Assignment\\test.json') as f:
    data = json.load(f)

# Print the data
print(data['startingState'])
startState = data['startingState'];
states = {key: value for key, value in data.items() if key != 'startingState'}
print("States:", states)

EPSION='EPS'

SA10
States: {'SA0': {'isTerminatingState': False, 'a': 'SB0'}, 'SB0': {'isTerminatingState': False, 'EPS': ['SA1']}, 'SA1': {'isTerminatingState': False, 'b': 'SB1'}, 'SB1': {'isTerminatingState': False, 'EPS': ['SB2']}, 'SA2': {'isTerminatingState': False, 'EPS': ['SA0']}, 'SB2': {'isTerminatingState': False, 'EPS': ['SA6']}, 'SA3': {'isTerminatingState': False, 'b': 'SB3'}, 'SB3': {'isTerminatingState': False, 'EPS': ['SB5']}, 'SA4': {'isTerminatingState': False, 'c': 'SB4'}, 'SB4': {'isTerminatingState': False, 'EPS': ['SB5']}, 'SA5': {'isTerminatingState': False, 'EPS': ['SA3', 'SA4']}, 'SB5': {'isTerminatingState': False, 'EPS': ['SA5', 'SB6']}, 'SA6': {'isTerminatingState': False, 'EPS': ['SA5', 'SB6']}, 'SB6': {'isTerminatingState': False, 'EPS': ['SB7']}, 'SA7': {'isTerminatingState': False, 'EPS': ['SA2']}, 'SB7': {'isTerminatingState': False, 'EPS': ['SA9']}, 'SA8': {'isTerminatingState': False, 'd': 'SB8'}, 'SB8': {'isTerminatingState': False, 'EPS': ['SA8', 'SB9']}, 'SA9':

In [4]:
def getClosure(startState, states):
    closure = []
    closure.append(startState)
    isAddedToClosure=True
    while isAddedToClosure:
        isAddedToClosure=False
        for item in closure:
            for key , value in states[item].items():
                if key== EPSION:
                    for item in value:
                        if item not in closure:
                            closure.append(item)
                            isAddedToClosure=True;
    return closure

In [5]:
class DFA:
    def __init__(self):
        self.startState = None
        self.acceptStates = []
        self.States = []
        self.transitions = {}

DFA = DFA()
DFA.startState = getClosure(startState, states)

st = []
st.append(DFA.startState)

while st:
    transMap = {}
    mainState = st.pop()
    mainState.sort()
    if mainState not in DFA.States:
        DFA.States.append(mainState)
    isAccept = False
    for item in mainState:
        for key, value in states[item].items():
            if key == "isTerminatingState" and value == True:
                isAccept = True
            if key != "isTerminatingState" and key !=EPSION:
                if key not in transMap:
                    transMap[key] = []
                #Check here if the value is a list or a single value
              
                transMap[key].append(value)
                value2 = getClosure(value, states)
                for Closureitem in value2:
                    transMap[key].append(Closureitem)
    if isAccept == True and mainState not in DFA.acceptStates:
        DFA.acceptStates.append(mainState)
    # Remove duplicates from each list in transMap
    for key in transMap:
        transMap[key] = list(set(transMap[key]))
        transMap[key].sort()
        if transMap[key] not in DFA.States:
            DFA.States.append(transMap[key])
            st.append(transMap[key])
        if tuple(mainState) not in DFA.transitions:
                DFA.transitions[tuple(mainState)] = {}
            
        DFA.transitions[tuple(mainState)][key] = transMap[key]    
print("DFA States:", DFA.States)
print("DFA Transitions:", DFA.transitions)        

DFA States: [['SA0', 'SA10', 'SA2', 'SA7'], ['SA1', 'SB0'], ['SA3', 'SA4', 'SA5', 'SA6', 'SA8', 'SA9', 'SB1', 'SB2', 'SB6', 'SB7'], ['SA3', 'SA4', 'SA5', 'SA8', 'SA9', 'SB3', 'SB5', 'SB6', 'SB7'], ['SA3', 'SA4', 'SA5', 'SA8', 'SA9', 'SB4', 'SB5', 'SB6', 'SB7'], ['SA8', 'SB10', 'SB8', 'SB9']]
DFA Transitions: {('SA0', 'SA10', 'SA2', 'SA7'): {'a': ['SA1', 'SB0']}, ('SA1', 'SB0'): {'b': ['SA3', 'SA4', 'SA5', 'SA6', 'SA8', 'SA9', 'SB1', 'SB2', 'SB6', 'SB7']}, ('SA3', 'SA4', 'SA5', 'SA6', 'SA8', 'SA9', 'SB1', 'SB2', 'SB6', 'SB7'): {'b': ['SA3', 'SA4', 'SA5', 'SA8', 'SA9', 'SB3', 'SB5', 'SB6', 'SB7'], 'c': ['SA3', 'SA4', 'SA5', 'SA8', 'SA9', 'SB4', 'SB5', 'SB6', 'SB7'], 'd': ['SA8', 'SB10', 'SB8', 'SB9']}, ('SA8', 'SB10', 'SB8', 'SB9'): {'d': ['SA8', 'SB10', 'SB8', 'SB9']}, ('SA3', 'SA4', 'SA5', 'SA8', 'SA9', 'SB4', 'SB5', 'SB6', 'SB7'): {'b': ['SA3', 'SA4', 'SA5', 'SA8', 'SA9', 'SB3', 'SB5', 'SB6', 'SB7'], 'c': ['SA3', 'SA4', 'SA5', 'SA8', 'SA9', 'SB4', 'SB5', 'SB6', 'SB7'], 'd': ['SA8', 'S

In [5]:
anotherMap = {}
count = 0
for item in DFA.States:
    anotherMap[tuple(item)] = 'S' + str(count)
    count += 1
DFA.startState = anotherMap[tuple(DFA.startState)]
# Convert DFA.States
for i in range(len(DFA.States)):
    DFA.States[i] = anotherMap[tuple(DFA.States[i])]

# Convert DFA.acceptStates
for i in range(len(DFA.acceptStates)):
    DFA.acceptStates[i] = anotherMap[tuple(DFA.acceptStates[i])]

# Convert DFA.transitions
new_transitions = {}
for key, value in DFA.transitions.items():
    new_key = anotherMap[tuple(key)]
    new_value = {k: anotherMap[tuple(v)] for k, v in value.items()}
    new_transitions[new_key] = new_value
DFA.transitions = new_transitions

In [6]:
#convert DFA to JSON
import json

# Create a dictionary that represents the DFA
dfa_dict = {
    "startingState": DFA.startState,
}

# Add states to the dictionary
for state in DFA.States:
    dfa_dict[state] = {
        "isTerminatingState": state in DFA.acceptStates,
    }
    # Add transitions for each state
    if state in DFA.transitions:
        for symbol, nextState in DFA.transitions[state].items():
            dfa_dict[state][symbol] = nextState

# Convert the dictionary to a JSON string
dfa_json = json.dumps(dfa_dict, indent=4)
# Specify the file path
file_path = 'B:/Spring2024/Compilers/Assignment/NFA_To_DFA.json'

# Open the file in write mode
with open(file_path, 'w') as f:
    # Write the JSON string to the file
    f.write(dfa_json)
dat2 = json.load(open('B:/Spring2024/Compilers/Assignment/NFA_To_DFA.json'))            
DFAstates = {key: value for key, value in dat2.items() if key != 'startingState'}
print(dfa_json)    

{
    "startingState": "S0",
    "S0": {
        "isTerminatingState": false,
        "a": "S1"
    },
    "S1": {
        "isTerminatingState": false,
        "b": "S2"
    },
    "S2": {
        "isTerminatingState": false,
        "b": "S3",
        "c": "S4",
        "d": "S5"
    },
    "S3": {
        "isTerminatingState": false,
        "b": "S3",
        "c": "S4",
        "d": "S5"
    },
    "S4": {
        "isTerminatingState": false,
        "b": "S3",
        "c": "S4",
        "d": "S5"
    },
    "S5": {
        "isTerminatingState": true,
        "d": "S5"
    }
}


In [7]:
class Min_DFA:
    def __init__(self):
        self.startState = None
        self.acceptStates = []
        self.States = []
        self.transitions = {}
Min_DFA = Min_DFA()

dfa_acceptStates = DFA.acceptStates
dfa_rejectStates=[]
dfa_state_to_min_dfa_state = {}
for state in DFA.States:
    if state not in dfa_acceptStates:
       dfa_rejectStates.append(state)
       dfa_state_to_min_dfa_state[state] = dfa_rejectStates
    else:
        dfa_state_to_min_dfa_state[state] = dfa_acceptStates
#2 bags created
min_dfa_states = [dfa_acceptStates, dfa_rejectStates]        

flag=True
while flag:
    flag=False
    for bag in min_dfa_states[:]:  # Create a copy for iteration
        if(len(bag)>1):
            split_map = []
            temp_map = {}
            for state in bag:
                transsitions_map={}
                for key, value in DFAstates[state].items():
                    if key != 'isTerminatingState':
                        if value not in transsitions_map:
                            transsitions_map[key] = []
                        transsitions_map[key].append(value)
                if transsitions_map not in split_map:
                    split_map.append(transsitions_map)
                    temp_map[state] = [split_map.index(transsitions_map)]  # Make it a list
                else:
                    if state not in temp_map:
                        temp_map[state] = []
                    temp_map[state].append(split_map.index(transsitions_map))  # Use append instead of add
            if len(split_map)>1:
                flag=True
                temp_map2 = {}
                for key,value in temp_map.items():  # Iterate over items, not the dictionary itself
                    if tuple(value) not in temp_map2:
                        temp_map2[tuple(value)] = []
                    temp_map2[tuple(value)].append(key)
                for value in temp_map2.values():  # Iterate over the values, not the keys
                    min_dfa_states.append(value)
                    for state in value:
                        dfa_state_to_min_dfa_state[state] = value
                min_dfa_states.remove(bag)
print(min_dfa_states)
print(dfa_state_to_min_dfa_state)


[['S5'], ['S0'], ['S1'], ['S2', 'S3', 'S4']]
{'S0': ['S0'], 'S1': ['S1'], 'S2': ['S2', 'S3', 'S4'], 'S3': ['S2', 'S3', 'S4'], 'S4': ['S2', 'S3', 'S4'], 'S5': ['S5']}


In [10]:
Min_DFA.startState = dfa_state_to_min_dfa_state[DFA.startState]

print(Min_DFA.acceptStates)


anotherMap2 = {}
count2 = 0
for key,value in dfa_state_to_min_dfa_state.items():
    if tuple(value) not in anotherMap2:
        anotherMap2[tuple(value)] = 'S' + str(count2)
        count2 += 1
print(anotherMap2)    
Min_DFA.startState=anotherMap2[tuple(Min_DFA.startState)]
for state in DFA.acceptStates:
    Min_DFA.acceptStates.append(anotherMap2[tuple(dfa_state_to_min_dfa_state[state])])
print(Min_DFA.startState)
for item in dfa_state_to_min_dfa_state:
    if anotherMap2[tuple(dfa_state_to_min_dfa_state[item])] not in Min_DFA.States:
        Min_DFA.States.append(anotherMap2[tuple(dfa_state_to_min_dfa_state[item])])
print(Min_DFA.States)

# # Convert DFA.transitions
new_transitions2 = {}
for key, value in DFA.transitions.items():
    new_key = anotherMap2[tuple(dfa_state_to_min_dfa_state[key])]
    new_value = {}  # Initialize an empty dictionary
    for k, v in value.items():
        tuple_key = tuple(dfa_state_to_min_dfa_state[v])
        anotherMap2_value = anotherMap2[tuple_key]
        new_value[k] = anotherMap2_value
    new_transitions2[new_key] = new_value
Min_DFA.transitions = new_transitions2    
print(new_transitions2)

[['S5']]
{('S0',): 'S0', ('S1',): 'S1', ('S2', 'S3', 'S4'): 'S2', ('S5',): 'S3'}
S0
['S0', 'S1', 'S2', 'S3']
{'S0': {'a': 'S1'}, 'S1': {'b': 'S2'}, 'S2': {'b': 'S2', 'c': 'S2', 'd': 'S3'}, 'S3': {'d': 'S3'}}


In [12]:
# Create a dictionary that represents the DFA
dfa_dict2 = {
    "startingState": Min_DFA.startState,
}

# Add states to the dictionary
for state in Min_DFA.States:
    dfa_dict2[state] = {
        "isTerminatingState": state in Min_DFA.acceptStates,
    }
    # Add transitions for each state
    if state in Min_DFA.transitions:
        for symbol, nextState in Min_DFA.transitions[state].items():
            dfa_dict2[state][symbol] = nextState

# Convert the dictionary to a JSON string
dfa_json2 = json.dumps(dfa_dict2, indent=4)
# Specify the file path
file_path = 'B:/Spring2024/Compilers/Assignment/Minmize_DFA.json'

# Open the file in write mode
with open(file_path, 'w') as f:
    # Write the JSON string to the file
    f.write(dfa_json2)
       

print(dfa_json2)    

{
    "startingState": "S0",
    "S0": {
        "isTerminatingState": false,
        "a": "S1"
    },
    "S1": {
        "isTerminatingState": false,
        "b": "S2"
    },
    "S2": {
        "isTerminatingState": false,
        "b": "S2",
        "c": "S2",
        "d": "S3"
    },
    "S3": {
        "isTerminatingState": true,
        "d": "S3"
    }
}
